---
---
---
# ***Recurrent Neural Network, RNN***
---
---
---

1. *Recurrent Neural Network, RNN*
2. *Long Short-Term Memory, LSTM*
3. *Gated Recurrent Unit, GRU*
4. *SimpleRNN과 LSTM*
5. *Recurrent Neural Network Language Model, RNNLM*
6. *Text Generation using RNN*

---
## 1. ***Recurrent Neural Network, RNN***
---

 - RNN은 대표적인 sequence model
 - DNN은 Feed Forward Neural Network 구조지만 RNN의 경우 Recurrent Neural Network 구조를 가짐
 - 즉, 출력층으로만 향하는 기존의 신경망과 달리 RNN은 출력층의 값이 다시 은닉층 노드로 향하는 순환신경망 구조


<br>

#### ***1. RNN***

 - RNN에서의 cell이란 ?
     - 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드
     - 이 셀은 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행하므로 이를 메모리 셀 또는 RNN 셀이라고 표현
     - 은닉층의 메모리 셀은 각각의 시점(time step)에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 하고 있음
     - 메모리 셀이 출력층 방향으로 또는 다음 시점 t+1의 자신에게 보내는 값을 은닉 상태(hidden state)라고 함

> RNN Structure <br>

![](https://wikidocs.net/images/page/22886/rnn_image2_ver3.PNG) <br>

> Detailed Structure <br>

![](https://wikidocs.net/images/page/22886/rnn_image2.5.PNG) <br>

> RNN Equation <br>

![](https://wikidocs.net/images/page/22886/rnn_image4_ver2.PNG)<br>
은닉층 : $h_t = tanh(w_x x_t + w_h h_{t-1} + b)$ <br>
출력층 : $y_t = f(w_y h_t + b)$

> RNN Input Shape <br>

![](https://wikidocs.net/images/page/22886/rnn_image6between7.PNG)

- How to print output state from cell?
    1. 메모리 셀의 최종 시점의 은닉 상태만을 리턴하고자 한다면 (batch_size, output_dim) 크기의 2D 텐서를 리턴
    2. 메모리 셀의 각 시점(time step)의 은닉 상태값들을 모아서 전체 시퀀스를 리턴하고자 한다면 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴
        - using `return_sequences = True`

In [1]:
# 함수 사용 없이 의사코드로 RNN 구현하기
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_dim = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_dim)) # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.
# print(hidden_state_t) 

Wx = np.random.random((hidden_size, input_dim))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0) 
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states) # (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.87137823 0.96849597 0.92462722 0.98108858 0.85341294 0.91668457
  0.96730208 0.79581152]
 [0.99970633 0.99991227 0.99996763 0.99950346 0.99974216 0.99925189
  0.99972503 0.99976492]
 [0.99985349 0.99994016 0.99998157 0.99977212 0.99983561 0.99965979
  0.99988174 0.99991575]
 [0.99993806 0.99998953 0.99999255 0.9999555  0.99997966 0.99990359
  0.99998081 0.99995623]
 [0.99992786 0.99998231 0.99999437 0.99992596 0.99997286 0.99984459
  0.99997404 0.9999584 ]
 [0.99994168 0.99999403 0.99999624 0.99993958 0.99996639 0.99987995
  0.99996377 0.99995486]
 [0.99992397 0.99998967 0.99999375 0.99987852 0.99997787 0.99982952
  0.99992822 0.9999152 ]
 [0.99990211 0.99996941 0.99999221 0.99985981 0.99994747 0.99975539
  0.99994473 0.99994252]
 [0.99995598 0.99999541 0.99999712 0.99996143 0.99999211 0.99991691
  0.99998413 0.99996154]
 [0.99989453 0.99997007 0.99998958 0.99985916 0.99992035 0.99976239
  0.99993181 0.99993547]

<br>

#### ***2. Bidirectional Recurrent Neural Network***
